In [2]:
#WebScraping
import time
import re
import io
from typing import Optional
from typing import Optional, Dict, List, Tuple

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/118.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Connection": "keep-alive",
}


def build_url(year: int) -> str:
    return f"https://www.spotrac.com/mlb/injured/_/year/{year}/view/player"


def fetch_html(url: str, max_retries: int = 3, backoff: float = 2.0) -> Optional[str]:
    for attempt in range(1, max_retries + 1):
        try:
            resp = requests.get(url, headers=HEADERS, timeout=20)
            if resp.status_code == 200 and "text/html" in resp.headers.get("Content-Type", ""):
                return resp.text
            if resp.status_code in (403, 429, 503):
                time.sleep(backoff * attempt)
                continue
            break
        except requests.RequestException:
            time.sleep(backoff * attempt)
    return None


def parse_table_with_pandas(html: str) -> Optional[pd.DataFrame]:
    try:
        tables = pd.read_html(io.StringIO(html))
        if not tables:
            return None
        return max(tables, key=lambda t: t.shape[1])
    except ValueError:
        return None


def parse_table_with_bs4(html: str) -> Optional[pd.DataFrame]:
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("table")
    if not table:
        return None

    thead = table.find("thead")
    if thead:
        headers = [th.get_text(strip=True) for th in thead.find_all("th")]
    else:
        first_row = table.find("tr")
        headers = [th.get_text(strip=True) for th in first_row.find_all(["th", "td"])] if first_row else []

    rows = []
    for tr in table.find_all("tr"):
        tds = tr.find_all("td")
        if not tds:
            continue
        rows.append([td.get_text(" ", strip=True) for td in tds])

    if not rows:
        return None

    max_len = max(len(r) for r in rows)
    if len(headers) != max_len:
        if len(headers) < max_len:
            headers += [f"col_{i+1}" for i in range(len(headers), max_len)]
        else:
            headers = headers[:max_len]

    return pd.DataFrame(rows, columns=headers)


def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [c.strip().replace("\n", " ").replace("  ", " ") for c in df.columns]
    df = df.dropna(axis=1, how="all")
    df = df.replace("", pd.NA).dropna(how="all")
    return df


def try_requests_then_playwright(url: str) -> pd.DataFrame:
    html = fetch_html(url)
    if html:
        for parser in (parse_table_with_pandas, parse_table_with_bs4):
            df = parser(html)
            if isinstance(df, pd.DataFrame) and not df.empty:
                return clean_df(df)

    try:
        from playwright.sync_api import sync_playwright
    except ImportError as e:
        raise RuntimeError(
            "Requests parsing failed and Playwright is not installed.\n"
            "Install with:\n  pip install playwright\n  playwright install chromium"
        ) from e

    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        context = browser.new_context(user_agent=HEADERS["User-Agent"])
        page = context.new_page()
        page.goto(url, wait_until="domcontentloaded", timeout=45000)
        page.wait_for_selector("table", timeout=20000)
        content = page.content()
        browser.close()

    for parser in (parse_table_with_pandas, parse_table_with_bs4):
        df = parser(content)
        if isinstance(df, pd.DataFrame) and not df.empty:
            return clean_df(df)

    raise RuntimeError("Could not locate a data table on the page after rendering.")


# ---------------- Normalizer ----------------

def _pick(df, *cands):
    cand_lc = [c.lower() for c in df.columns]
    for want in cands:
        for i, c in enumerate(cand_lc):
            if want in c:
                return df.columns[i]
    return None


def normalize_spotrac_injured_df(raw: pd.DataFrame) -> pd.DataFrame:
    """
    Fixed version: correct parsing of multiple IL stints.
    """
    df = raw.copy()

    col_rank = _pick(df, "rank")
    col_player = _pick(df, "player")
    col_pos = _pick(df, "pos")
    col_team = _pick(df, "team")
    col_reason = _pick(df, "reason")

    if any(c is None for c in [col_player, col_reason]):
        raise ValueError(f"Missing required columns. Found: {list(df.columns)}")

    def _clean(s):
        return s.astype(str).str.replace(r"\s+", " ", regex=True).str.strip()

    if col_rank:
        df[col_rank] = pd.to_numeric(df[col_rank], errors="coerce").astype("Int64")

    df[col_player] = _clean(df[col_player])
    if col_pos: df[col_pos] = _clean(df[col_pos])
    if col_team: df[col_team] = _clean(df[col_team])

    # *** NEW FIXED REGEX (matches each IL block independently) ***
    rx_entry = re.compile(
        r"""
        (?P<il>[^:,]+?(?:IL|List|Suspension|Restricted(?:\s+List)?))
        (?:\s*-\s*(?P<inj>[^:,]+?))?
        \s*:\s*
        (?P<start>\d{1,2}/\d{1,2}/\d{2})
        \s*-\s*
        (?P<end>\d{1,2}/\d{1,2}/\d{2})
        """,
        re.IGNORECASE | re.VERBOSE
    )

    records = []

    for _, row in df.iterrows():
        base = {
            "rank": row[col_rank] if col_rank else pd.NA,
            "player": row[col_player],
            "pos": row[col_pos] if col_pos else pd.NA,
            "team": row[col_team] if col_team else pd.NA,
        }

        text = str(row[col_reason])

        matches = list(rx_entry.finditer(text))

        if matches:
            for m in matches:
                rec = dict(base)
                rec["il_type"] = (m.group("il") or "").strip()
                rec["injury"] = (m.group("inj") or "").strip()
                rec["start_date"] = pd.to_datetime(m.group("start"), format="%m/%d/%y", errors="coerce")
                rec["end_date"] = pd.to_datetime(m.group("end"), format="%m/%d/%y", errors="coerce")
                rec["reason_raw"] = text
                records.append(rec)
        else:
            # fallback single-entry parser
            rec = dict(base)
            rec["il_type"] = ""
            rec["injury"] = ""
            rec["start_date"] = pd.NaT
            rec["end_date"] = pd.NaT
            rec["reason_raw"] = text
            records.append(rec)

    out = pd.DataFrame.from_records(records)

    want = [
        "rank", "player", "pos", "team",
        "il_type", "injury", "start_date", "end_date",
        "reason_raw"
    ]
    return out[[c for c in want if c in out.columns]]


# ---------------- Multi-year API ----------------

def scrape_spotrac_years(years: List[int], sleep_sec: float = 1.0) -> Tuple[Dict[int, pd.DataFrame], pd.DataFrame]:
    tables = {}
    frames = []
    for yr in years:
        url = build_url(yr)
        raw = try_requests_then_playwright(url)
        clean = normalize_spotrac_injured_df(raw)
        clean["year"] = yr
        tables[yr] = clean
        frames.append(clean)
        if sleep_sec:
            time.sleep(sleep_sec)

    combined = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    return tables, combined

In [4]:
# years = [2015, 2020, 2021, 2022, 2023, 2024, 2025]

[i for i in range(2018, 2026)]

tables, combined = scrape_spotrac_years([i for i in range(2018, 2026)])

# Inspect
{y: df.shape for y, df in tables.items()}
combined

,rank,player,pos,team,il_type,injury,start_date,end_date,reason_raw,year
0,1,Miguel Cabrera,DH,DET,10-Day IL,Hamstring,2018-05-04,2018-06-01,10-Day IL - Hamstring: 5/4/18-6/1/18 10-Day I...,2018
1,1,Miguel Cabrera,DH,DET,10-Day IL,Biceps,2018-06-13,2018-10-01,10-Day IL - Hamstring: 5/4/18-6/1/18 10-Day I...,2018
2,2,Jacoby Ellsbury,CF,NYY,60-Day IL,Oblique,2018-03-29,2018-10-01,60-Day IL - Oblique: 3/29/18-10/1/18,2018
3,3,Yoenis Céspedes,LF,NYM,10-Day IL,Hip,2018-05-16,2018-07-20,10-Day IL - Hip: 5/16/18-7/20/18 60-Day IL - ...,2018
4,3,Yoenis Céspedes,LF,NYM,60-Day IL,Heel,2018-07-24,2018-10-01,10-Day IL - Hip: 5/16/18-7/20/18 60-Day IL - ...,2018
...,...,...,...,...,...,...,...,...,...,...
6332,628,Cade Horton,SP,CHC,15-Day IL,Ribs,2025-09-25,2025-09-28,15-Day IL - Ribs: 9/25/25-9/28/25,2025
6333,629,Everson Pereira,OF,TB,10-Day IL,Back,2025-09-26,2025-09-28,10-Day IL - Back: 9/26/25-9/28/25,2025
6334,630,Brett Baty,2B,NYM,10-Day IL,Oblique,2025-09-27,2025-09-28,10-Day IL - Oblique: 9/27/25-9/28/25,2025
6335,631,Nick Frasso,SP,LAD,60-Day IL,Undisclosed,2025-09-27,2025-09-28,60-Day IL - Undisclosed: 9/27/25-9/28/25,2025


In [6]:
combined[combined['player'] == 'Adrian Morejon']

,rank,player,pos,team,il_type,injury,start_date,end_date,reason_raw,year
1177,389,Adrian Morejon,RP,SD,60-Day IL,Shoulder,2019-08-07,2019-09-29,60-Day IL - Shoulder: 8/7/19-9/29/19,2019
2290,264,Adrian Morejon,RP,SD,60-Day IL,Arm,2021-04-12,2021-10-03,60-Day IL - Arm: 4/12/21-10/3/21,2021
3736,520,Adrian Morejon,RP,SD,60-Day IL,Elbow Tommy John,2022-04-04,2022-06-06,60-Day IL - Elbow Tommy John: 4/4/22-6/6/22 1...,2022
3737,520,Adrian Morejon,RP,SD,10-Day IL,Covid 19,2022-06-11,2022-06-16,60-Day IL - Elbow Tommy John: 4/4/22-6/6/22 1...,2022
3738,520,Adrian Morejon,RP,SD,15-Day IL,Shoulder,2022-06-23,2022-07-14,60-Day IL - Elbow Tommy John: 4/4/22-6/6/22 1...,2022
4402,328,Adrian Morejon,RP,SD,60-Day IL,Elbow,2023-03-30,2023-06-06,60-Day IL - Elbow: 3/30/23-6/6/23 15-Day IL -...,2023
4403,328,Adrian Morejon,RP,SD,15-Day IL,Knee,2023-07-16,2023-08-17,60-Day IL - Elbow: 3/30/23-6/6/23 15-Day IL -...,2023


In [7]:
combined.to_csv("mlb_injuries.csv", index=False)
# # or per year:
# for y, df in tables.items():
#     df.to_parquet(f"mlb_injured_{y}.parquet", index=False)